## Декоратор @Logger

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [1]:
import sys
import datetime
import functools

In [2]:
def singleton(cls):
    instance = None
    
    @functools.wraps(cls)
    def wrapper(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    return wrapper

In [3]:
@singleton
class SingleLogger:
    
    def __init__(self):
        self.index = 0
    
    def __call__(self, func, dest, *args, **kwargs):
        if dest in [sys.stdout, sys.stderr]:
            self.out = dest
        else:
            self.out = open(dest, 'a')
            
        try:
            func_out = func(*args, **kwargs)
        except Exception as e:
            self._write(func.__name__, str(e), *args, **kwargs)
            raise
        self._write(func.__name__, func_out, *args, **kwargs)
        self.index += 1
        return func_out
    
    def __del__(self):
        self.out.close()
        
    def _write(self, func_name, func_out, *args, **kwargs):
        self.out.write('index: {} datetime: {} func_name: {} args: {} kwargs: {} result: {}'.format(
            self.index, datetime.datetime.now(), func_name, args, kwargs, func_out))

In [4]:
def logger(dest):
    log = SingleLogger()
    
    def wrapped(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return log(func, dest, *args, **kwargs)
        return wrapper
    return wrapped

Проверим работу декоратора.

In [5]:
@logger(sys.stdout)
def test_stdout(a, b, c):
    return str(a) + ' ' + str(b) + ' ' + str(c) 

In [6]:
res = test_stdout('I', 2, 3)

index: 0 datetime: 2020-05-12 20:01:26.688315 func_name: test_stdout args: ('I', 2, 3) kwargs: {} result: I 2 3

In [7]:
@logger(sys.stderr)
def test_stderr(*args, **kwargs):
    return len(args)

In [8]:
res = test_stderr(2, 5, 7)

index: 1 datetime: 2020-05-12 20:01:26.700488 func_name: test_stderr args: (2, 5, 7) kwargs: {} result: 3

In [9]:
@logger('test.txt')
def test_file(*args, **kwargs):
    return len(kwargs)

In [10]:
res = test_file(a=1, b=3)
print(res)

2
